In [16]:
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from threading import Thread
import threading
lock = threading.RLock()

In [12]:
talks = pd.read_pickle('../datasets/videosData.pkl')
talks['views'] = talks.views.astype(int)
talks['likes'] = talks.likes.astype(int)
talks['dislikes'] = talks.dislikes.astype(int)
print(len(talks))
talks.head(2)

def getEvent(x):
    matchs = re.findall(r'(TEDx[^! ]+|TEDx [^! ]+)', x, re.IGNORECASE)
    if(len(matchs) > 0):
        return matchs[0].replace(' ','')
    else:
        return np.nan

talks['event'] = talks.titleRaw.apply(getEvent)
talks = talks[~pd.isna(talks.event)]

78391


In [13]:
eventsDf = pd.read_pickle('../datasets/eventsParsed.pkl')
eventsDf = eventsDf.drop_duplicates(subset='name')
print(len(eventsDf), 'events')
eventsDf.index = eventsDf.name
eventsDf.head(5)

7285 events


,name,eventType,date,city,country,hasWebcast,hasSpace,eventCode,shortName
name,,,,,,,,,
TEDxRiodelaPlata,TEDxRiodelaPlata,Standard,"May 7, 2019",Ciudad Autónoma de Buenos Aires,Argentina,True,False,32790,RiodelaPlata
TEDxEsquel,TEDxEsquel,Standard,"May 4, 2019",Esquel,Argentina,True,False,33511,Esquel
TEDxRafaela,TEDxRafaela,Standard,"May 4, 2019",Rafaela,Argentina,True,False,31229,Rafaela
TEDxBarilocheLive,TEDxBarilocheLive,TEDxLive,"April 27, 2019",San Carlos de Bariloche,Argentina,True,False,34453,Bariloche
TEDxRosarioLive,TEDxRosarioLive,TEDxLive,"April 18, 2019",Rosario,Argentina,True,False,34383,Rosario


In [14]:
targets = talks[~talks.event.isin(eventsDf.index.values)].event.unique()

In [17]:
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])

proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
proxiesDf.head()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
0,185.61.94.65,61616,RU,Russian Federation,elite proxy,no,yes,2 minutes ago
37,140.227.202.194,6000,JP,Japan,elite proxy,no,yes,3 minutes ago
57,5.63.165.178,40561,AM,Armenia,elite proxy,no,yes,3 minutes ago
56,177.128.42.25,50041,BR,Brazil,elite proxy,no,yes,3 minutes ago
54,151.253.165.70,8080,AE,United Arab Emirates,elite proxy,no,yes,3 minutes ago


In [18]:
eventsDf = pd.read_pickle('../datasets/rawEvents.pkl')
print(len(eventsDf), 'events')
eventsDf.head()

12998 events


,name,eventType,date,city,country,hasWebcast,hasSpace,eventCode
32790,TEDxRiodelaPlata,Standard,"May 7, 2019",Ciudad Autónoma de Buenos Aires,Argentina,True,False,32790
33511,TEDxEsquel,Standard,"May 4, 2019",Esquel,Argentina,True,False,33511
31229,TEDxRafaela,Standard,"May 4, 2019",Rafaela,Argentina,True,False,31229
34453,TEDxBarilocheLive,TEDxLive,"April 27, 2019",San Carlos de Bariloche,Argentina,True,False,34453
34383,TEDxRosarioLive,TEDxLive,"April 18, 2019",Rosario,Argentina,True,False,34383


In [27]:
errors = pd.DataFrame(columns=['page','min','max','country'])
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf[proxiesDf.Https=='no'].loc[:,'IP Address'].values[0]
}
def getProxy():
    return {
        "http": proxiesDf[proxiesDf.Https=='no'].loc[:,'IP Address'].sample(1).values
    }
def getUrl(url):
    return requests.get(
        url, headers=headersChrome, proxies=getProxy()
    )

def getEvents(searchFilter, startPage=1, maxPages=1, past=False):
    whenStr = 'upcoming'
    if(past):
        whenStr = 'past'
    if(type(maxPages) is not int):
        print('maxPages has to be int')
        return False
    for pageNum in range(startPage, maxPages+1):
#         searchFilter = urllib.parse.quote(searchFilter)
        url = 'https://www.ted.com/tedx/events?autocomplete_filter='+searchFilter+'&page='+str(pageNum)+'&when='+whenStr
        display(url)
        rawHtmlPage = getUrl(url)
        html = BeautifulSoup(rawHtmlPage.text, 'html.parser')
        eventsRows = html.select('tr.tedx-events-table__event')
        if(len(eventsRows) == 0):
            break
        for x in eventsRows:
            locationRaw = x.select('td')[2]
            ciudad, pais = re.findall('(.+)<br\/>(.+)<\/td>', str(locationRaw))[0]
            dateRaw = x.select('td:nth-of-type(1)')[0].text
            dateStr = re.findall('(\w+ [\d]+, [\d]+)', dateRaw)[0]
            a = x.select('strong a')[0]
            href = a['href']
            name = re.sub('\xad','', str(a.text))
            webcastRaw = x.select('td')[4]
            hasWebcast = re.findall('(.+)', str(webcastRaw))[3]!='No'

            spaceRaw = x.select('td')[3]
            hasSpace = re.findall('<span class="screen-reader-text">(.+)</span>', str(spaceRaw))[0] == 'Yes'
            eventCode = int(re.sub('/tedx/events/','', href))
            eventType = x.find_all(attrs={'class':'tedx-events-table__event-type'})[0].text
            with lock:
                eventsDf.loc[eventCode] = [
                    name,
                    eventType,
                    dateStr,
                    ciudad,
                    pais,
                    hasWebcast,
                    hasSpace,
                    eventCode,
                ]
    return True

getEvents('TEDxLincolnSchool', 1, 100, True)

'https://www.ted.com/tedx/events?autocomplete_filter=TEDxLincolnSchool&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=TEDxLincolnSchool&page=2&when=past'

True

In [30]:
targets[:10]

array(['TEDxRíodelaPlata', 'TEDxBahíaBlanca', 'TEDxLincolnSchool',
       'TEDxUFM', 'TEDxParqueElEjidoMujeres', 'TEDxPanamaCity',
       'TEDxCiudadDeCorrientes', 'TEDxperiments:', 'TEDxITA',
       'TEDxKamitiPrison'], dtype=object)

In [33]:
isPast = True
def process_id(country):
    """process a single ID"""
    global isPast
    country = re.sub(' ', '+', country)
    try:
        getEvents(country, 1, 1000, isPast)
    except:
        print('ERROR IN', country)
    return True

def process_range(id_range, store=None):
    """process a number of ids, storing the results in a dict"""
    if store is None:
        store = {}
    for id in id_range:
        store[id] = process_id(id)
    return store

from threading import Thread

def threaded_process_range(nthreads, id_range):
    """process the id range in a specified number of threads"""
    store = {}
    threads = []
    # create the threads
    for i in range(nthreads):
        ids = id_range[i::nthreads]
        t = Thread(target=process_range, args=(ids,store))
        threads.append(t)

    # start the threads
    [ t.start() for t in threads ]
    # wait for the threads to finish
    [ t.join() for t in threads ]
    return store

# countries = eventsDf.country.unique()
threaded_process_range(
    3,
    targets,
)

'https://www.ted.com/tedx/events?autocomplete_filter=TEDxLincolnSchool&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=TEDxRíodelaPlata&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=TEDxBahíaBlanca&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=TEDxLincolnSchool&page=2&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=TEDxBahíaBlanca&page=2&when=past'

...

'https://www.ted.com/tedx/events?autocomplete_filter=TEDxBaghdad:&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=TEDxJU&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=TEDxJU&page=2&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=TEDxBMIHMS:&page=1&when=past'

{'TEDxLincolnSchool': True,
 'TEDxBahíaBlanca': True,
 'TEDxRíodelaPlata': True,
 'TEDxPanamaCity': True,
 'TEDxParqueElEjidoMujeres': True,
 'TEDxperiments:': True,
 'TEDxUFM': True,
 'TEDxCiudadDeCorrientes': True,
 'TEDxITA': True,
 'TEDxKamitiPrison': True,
 'TEDxKalamata': True,
 'TEDxKeioHighSchool': True,
 'TEDxGodaPark': True,
 'TEDxYouth@Dorado': True,
 'TEDxChaoyangWomen': True,
 'TEDxUWCSEA': True,
 'TEDxYouth@Quito': True,
 'TEDxYouth@Winchester': True,
 'TEDxYouth@KulibinPark': True,
 'TEDxAruba': True,
 'TEDxPelourinho': True,
 'TEDxYouth@Kolmikulma': True,
 'TEDxLESEYNE': True,
 'TEDxSoweto': True,
 'TEDxEdgemontSchool': True,
 'TEDxSkoll': True,
 'TEDxTangsel': True,
 'TEDxTughlaqRd': True,
 'TEDxYouth@Edmonton': True,
 'TEDxUKZN': True,
 'TEDxTalk': True,
 'TEDxHolborn': True,
 'TEDxNewEngland': True,
 'TEDxCaltech': True,
 'TEDxAnogeia': True,
 'TEDxOtaniemiED': True,
 'TEDxUWCCR': True,
 'TEDxSquareMile2013': True,
 'TEDxCórdoba': True,
 'TEDxYouth@SPCP': True,
 'TED

In [34]:
print(len(eventsDf), 'events')
eventsDf.head()

20217 events


,name,eventType,date,city,country,hasWebcast,hasSpace,eventCode
32790,TEDxRiodelaPlata,Standard,"May 7, 2019",Ciudad Autónoma de Buenos Aires,Argentina,True,False,32790
33511,TEDxEsquel,Standard,"May 4, 2019",Esquel,Argentina,True,False,33511
31229,TEDxRafaela,Standard,"May 4, 2019",Rafaela,Argentina,True,False,31229
34453,TEDxBarilocheLive,TEDxLive,"April 27, 2019",San Carlos de Bariloche,Argentina,True,False,34453
34383,TEDxRosarioLive,TEDxLive,"April 18, 2019",Rosario,Argentina,True,False,34383


In [35]:
eventsDf.to_pickle('../datasets/rawEvents.pkl')